# **Cinematica inversa de un brazo robotico**

La posicion del extremo del brazo en el plano esta dada por

$$
\begin{align*}
x(\theta_1,\theta_2) = L_1 \cos(\theta_1) + L_2 \cos(\theta_1 + \theta_2) \\
y(\theta_1,\theta_2) = L_1 \sin(\theta_1) + L_2 \sin(\theta_1 + \theta_2)
\end{align*}

$$


<center>

![alt text](./images/brazo-robotico.png)
</center>

Para alcanzar un punto deseado $(x_d, y_d)$ se plantea el siguiente sistema

$$
\begin{align*}
F_1(\theta_1,\theta_2) = x(\theta_1,\theta_2) - x_d = 0 \\
F_2(\theta_1,\theta_2) = y(\theta_1,\theta_2) - y_d = 0
\end{align*}
$$

## **Algorithmic implementation**

In order to have an algorithm for Newton's method for non-linear systems of equations, the idea is to create implementation accepts the least amount of parameters such that it can perform all calculations required.

But before that we must define the preliminars needed. Let us consider the following non-linear system of equations with $n$ variables and $n$ equations:

$$
\begin{gathered}
f_1(x_1,x_2,\dots,x_n) = 0 \\
f_2(x_1,x_2,\dots,x_n) = 0 \\
\vdots \\
f_n(x_1,x_2,\dots,x_n) = 0
\end{gathered}
$$

We can solve this system by defining a vector field that depends in a single vector, where each component represents a variable of the system.

$$
\begin{align*}
\boldsymbol{F}(\boldsymbol{X}) = \boldsymbol{0}, \quad \text{where } &\boldsymbol{X} = (x_1, x_2, \dots, x_n)^T \\
&\boldsymbol{F} = (f_1, f_2, \dots, f_n)^T
\end{align*}
$$

This way we can define the standard equation for Newton's method but in a vectorish way

$$
\begin{align*}
\boldsymbol{X^{(k+1)}} = \boldsymbol{X^{(k)}} - \boldsymbol{J}\left(\boldsymbol{X^{(k)}}\right)^{-1} \cdot \boldsymbol{F}\left(\boldsymbol{X^{(k)}}\right) 
\end{align*}
$$

In [ ]:
from typing import Callable, List, Tuple, TypedDict, Union

import autograd.numpy as anp
import numpy as np
import numpy.typing as npt
from autograd import jacobian

np.set_printoptions(suppress=True)


class NewtonResult(TypedDict):
    solution: npt.NDArray[np.floating]
    message: str


def newton_vector(
    F: Callable[[npt.NDArray[np.floating]], npt.NDArray[np.floating]],
    x0: Union[npt.ArrayLike, List[float], Tuple[float]],
    delta: float,
    m: int,
) -> NewtonResult:
    J = jacobian(F)
    x = np.asarray(x0, dtype=float)

    for i in range(m):
        x = x - np.linalg.inv(J(x)) @ F(x)
        if np.linalg.norm(F(x)) < delta:
            return {
                "solution": x,
                "message": f"Convergence satisfied after {i+1} iterations",
            }

    return {
        "solution": x,
        "message": "Maximum number of iterations exceeded",
    }